In [28]:
# Data processing  
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import pickle

# Pandas options  
# -----------------------------------------------------------------------
pd.options.display.max_colwidth = None

# Path configuration for custom module imports
# -----------------------------------------------------------------------
import sys
sys.path.append('../')  # Adds the parent directory to the path for custom module imports

# Ignore warnings  
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# Custom functions and classes
# -----------------------------------------------------------------------
from src.support_encoding import Encoding, chi2_test
from src.support_scaling import scale_df
from src.support_eda import value_counts

## Data loading

In [14]:
df = pd.read_csv('../data/output/complete_data_imputed.csv', index_col=0).reset_index(drop=True)

In [15]:
df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,...,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,Female,1,Healthcare Representative,...,11,0,1.0,6,1,0,3.0,4.0,2.0,3
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,Female,1,Research Scientist,...,23,1,6.0,3,5,1,3.0,2.0,4.0,2
2,32,No,Travel_Frequently,Research & Development,17,4,Other,Male,4,Sales Executive,...,15,3,5.0,2,5,0,2.0,2.0,1.0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,Male,3,Human Resources,...,11,3,13.0,5,8,7,4.0,4.0,3.0,2
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,Male,1,Sales Executive,...,12,2,9.0,2,6,0,4.0,1.0,3.0,3


## Chi2 test

In [16]:
catgs = df.select_dtypes(include=['O', 'category']).columns

In [17]:
chi2_test(df,  catgs, 'Attrition', show=True)

We are evaluating the variable ATTRITION


Attrition,No,Yes
Attrition,,
No,3699,0
Yes,0,711


For the category ATTRITION there are significant differences, p = 0.0000


Attrition,No,Yes
Attrition,,
No,3103.0,596.0
Yes,596.0,115.0


--------------------------
We are evaluating the variable BUSINESSTRAVEL


...

--------------------------
We are evaluating the variable MARITALSTATUS


Attrition,No,Yes
MaritalStatus,,
Divorced,882,99
Married,1767,252
Single,1050,360


For the category MARITALSTATUS there are significant differences, p = 0.0000


Attrition,No,Yes
MaritalStatus,,
Divorced,823.0,158.0
Married,1693.0,326.0
Single,1183.0,227.0


--------------------------


## Encoding

Lo primero que vamos a hacer es convertir la target varible, `Attrition`, a numérica de la siguiente forma:

* `No`: `0`

* `Yes`: `1`

In [18]:
df['Attrition'].replace({'Yes': 1, 'No': 0}, inplace=True)

In [19]:
df.select_dtypes(include=['O', 'category']).head()

,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus
0,Travel_Rarely,Sales,Life Sciences,Female,Healthcare Representative,Married
1,Travel_Frequently,Research & Development,Life Sciences,Female,Research Scientist,Single
2,Travel_Frequently,Research & Development,Other,Male,Sales Executive,Married
3,Non-Travel,Research & Development,Life Sciences,Male,Human Resources,Married
4,Travel_Rarely,Research & Development,Medical,Male,Sales Executive,Single


Hay diferencias significativas en todas las variables salvo en `Gender`, por lo que usaremos `TargetEncoding` en todas salvo en género, que utilizaremos `OneHot`

In [20]:
encoding_methods = {"onehot": ['Gender'],
                    "target": ['BusinessTravel', 'Department', 'EducationField', 'JobRole', 'MaritalStatus'],
                    "ordinal" : {},
                    "frequency": []
                    }

encoder = Encoding(df, encoding_methods, 'Attrition')

In [21]:
df_encoded = encoder.execute_all_encodings()

### Scaling

Como hemos utilizado `OneHotEncoder` y `TargetEncoder` hace target a la variable binaria `Attrition` vale la pena comenzar probando a utilizar un `MinMaxScaler` ya que tampoco ha parecido haber demasiados outliers.

In [25]:
df_scaled, scaler = scale_df(df_encoded, df_encoded.columns.to_list(), method="minmax")

### Imbalanced data

In [29]:
value_counts(df, 'Attrition')

The number of unique values for this category is 2


,count,proportion
Attrition,,
0,3699,0.84
1,711,0.16


Nuestro conjunto de datos está bastante desbalanceado, por lo que en futuros modelos podríamos considerar un rebalanceo mediante remuestreo

### Save data

In [27]:
df_scaled.to_csv('../data/output/complete_data_preprocessed.csv')